In [1]:
#Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

import os
import signal
import sys
# Configuraciones
import re
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium.common.exceptions import NoSuchElementException

In [2]:

#Creamos una función para poder ir guardando la info en caso de interrupción 
def signal_handler(signal, frame, driver, datos_peliculas):
    print("Proceso interrumpido. Guardando datos...")
    df_resultados_iniciales = pd.DataFrame(datos_peliculas)
    df_resultados_iniciales.to_csv('datos_iniciales.csv', index=False)
    print("Datos guardados en 'datos_iniciales.csv'.")
    guardar_datos(datos_peliculas)  #  definir esta función
    driver.close()
    sys.exit(0)
    
def guardar_datos(datos_peliculas):
   # Definimos el  archivo donde vamos a guardar los datos.
    ruta_archivo = 'fase-3-datos-actorxs-.csv'
    
    # Convertimos el diccionario a DataFrame
    df_nuevos_datos = pd.DataFrame(datos_peliculas)
    
    if os.path.exists(ruta_archivo):
        # Si el archivo ya existe, cargamos los datos existentes y los juntamos.
        df_existentes = pd.read_csv(ruta_archivo)
        df_resultante = pd.concat([df_existentes, df_nuevos_datos], ignore_index=True)
    else:
        # Si el archivo no existe, utiliza los nuevos datos directamente
        df_resultante = df_nuevos_datos
    
    # Guardamos el DataFrame resultante en el archivo, sobrescribiendo el archivo existente
    df_resultante.to_csv(ruta_archivo, index=False)
    
    print(f"Datos guardados o actualizados exitosamente en {ruta_archivo}.")

#definimos una función de try y except para los xpath, busca los datos con el xpath y si no lo encuentra, añade "N/A"
def get_data(driver,xpath): 
    try:
        data = driver.find_element("xpath", xpath).text
    except:
        data = "N/A"
    return data

#Creamos la función de extración de datos de los actores
def extraer_datos_actores(nombre_archivo): 

    df_pelis = pd.read_csv(nombre_archivo) #Extraemos la info de las películas de IMDb y nos quedamos con el ID de cada una

    df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str) #Igualamos los ID para que sean todos strings
    #Creamos una lista vacía
    datos_actores_total = []
    
    driver = webdriver.Chrome() #abrimos el chrome
    driver.maximize_window() #Lo maximizamos

    signal.signal(signal.SIGINT, lambda sig, frame: signal_handler(sig, frame, driver, datos_peliculas))  #Esto es para poder interrumpir la función.

    for id_imdb in df_pelis["ID de la Película"][801:1000]: #Iteramos sobre los ID de cada película
        try:
            url = f"https://www.imdb.com/title/{id_imdb}/" #cargamos la página de IMDb en Chrome
            driver.get(url)

            sleep(2)
            try:
                driver.find_element(
                    "xpath", '//*[@id="__next"]/div/div/div[2]/div/button[2]').click() #aceptamos las cookies
            except NoSuchElementException:
                pass

            sleep(2)  
            #hacemos clic en "reparto y equipo"
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/ul/li[1]/a').click() 

            sleep(2)

            lista_actores = [] #creamos un listado vacío dentro del bucle para guardar la info

            # Buscamos todos los actores de la película
            actores = driver.find_elements(
                'class name', 'cast_list')[0].find_elements('tag name', 'tr')
            
            # aquí contamos cuántos actores hay en la lista. Le restamos 1 porque el primer elemento es el encabezado de la tabla y no tiene datos.
            numero_maximo_actores = 10 if len(
                actores)-1 > 10 else len(actores)-1

            #sacamos el nombre de la película
            nombre_peli = get_data(driver, '//*[@id="main"]/div[1]/div[1]/div/div[1]/h3/a') 

            for i in range(2, numero_maximo_actores): #Iteramos para extraer la info de los primeros 10 actores
                try:

                    sleep(2)
                     # Hacemos clic en el enlace del actor/actriz para que nos lleve a su página
                    driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
                    sleep(3) 
                    #Cogemos su nombre
                    nombre_actxr = get_data(driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span') #Extraemos nombre actor/actrix

                    #Extraemos las películas por las que estos actores son conocidos
                    conocido_por1 = get_data(driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/a')

                    conocido_por2 = get_data(driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[2]/div[2]/div[1]/a')
                    conocido_por = [
                        conocido_por1, conocido_por2] if conocido_por2 != "N/A" else [conocido_por1]

                    # Extraemos la fecha de nacimiento haciendo un slicing para coger simplemente el año de nacimiento, y manejamos la excepción: de no ser un año poner N/A
                    fecha_nacimiento = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[2]/section/aside/div/span[2]')
                    year_nacimiento = fecha_nacimiento.split(
                        " ")[-1:][0] if fecha_nacimiento != "N/A" else "N/A"

                    # Extraemos las profesiones del actor y moldeamos el texto para que si tiene más de una, aparezcan separadas por comas.
                    profesiones = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul')
                    profesiones_string = ", ".join(
                        profesiones.split("\n")) if profesiones != "N/A" else "N/A"

                    # Cogemos los premios del actor, si es que tiene
                    premios = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span')
                    
                    #añadimos todos los datos extraídos a nuestra lista vacia
                    info_actores = (nombre_peli, nombre_actxr, conocido_por, #añadimos todos los datos extraídos a nuestra lista vacia
                                    year_nacimiento, profesiones_string, premios)
                    lista_actores.append(info_actores)
                    
                #Manejamos la excepción si no hay 10 actores o no encuentra alguno, para que nos diga cuál ha sido
                except NoSuchElementException:
                    print(f"\033[95mNo se encontró el actor número {i-1}\033[0m") 
                    pass

                finally:
                    driver.back() #volvemos para atrás para ir al siguiente actor

            print("Datos de los actores:")
            print(lista_actores)
            #Fuera del bucle, añadimos la lista con la info de cada actor/actriz a la lista con toda la información.
            datos_actores_total.extend(lista_actores)
        except Exception as e:
            print(f"Error al procesor la película {nombre_peli} con ID de IMDb {id_imdb}: {e}") #En caso de que no haya podido sacar datos de actores, que nos diga de que pelicula
    
    #Cerramos el driver
    driver.quit()
    #Le decimos que nos devuelva totos los datos de los actores.
    return datos_actores_total


In [3]:
#Leemos el csv con todas las peliculas, que contiene sus ID´s de IMDB
df_peliculas = pd.read_csv("peliculas_api.csv") 

In [4]:
# Llamamos a  la función con nuestro archivo de pelis que sacamos de la API.
extraccion = extraer_datos_actores("peliculas_api.csv")

Datos de los actores:
[]
No se encontró el actor número 7
No se encontró el actor número 8
Datos de los actores:
[('Redemption', 'Derek Burke', ['Redemption', 'Reversion'], 'N/A', 'Reparto', 'N/A'), ('Redemption', 'July Smith', ['Redemption', 'A Fire in a Dovecot'], 'N/A', 'Reparto', 'N/A'), ('Redemption', 'Adam Chambers', ['Skylar contra el monstruo', 'I <3 Vampires'], '1983', 'Reparto, Dirección, Producción', '3 premios y 2 nominaciones'), ('Redemption', 'Shaina Vorspan', ['Dollface', 'Morir de pie'], 'junio', 'Reparto, Guion, Dirección', '1 premio y 1 nominación'), ('Redemption', 'Marco Neves', ['4 Horsemen: Apocalypse', 'Redemption'], 'N/A', 'Reparto, Equipos adicionales', '1,76 m'), ('Redemption', 'Aníbal Silveyra', ['Generation P', 'El año de la Marmota'], 'N/A', 'Reparto, Dirección', 'Malena')]
Datos de los actores:
[]
Datos de los actores:
[('Final Level', 'Oneira Acosta', ['Final Level'], 'N/A', 'Reparto', 'N/A'), ('Final Level', 'Kelly Adames', ['Final Level'], 'N/A', 'Repart

In [5]:
#Pasamos a dataframe la lista de listas, definiendo el nombre de sus columnas
df_extraccion = df = pd.DataFrame(extraccion, columns=["nombre_peli", "nombre_actxr", "conocido_por", "year_nacimiento", "profesiones_string", "premios"]) #creación del dataframe

In [6]:
#Visualizamos el Dataframe
df_extraccion

,nombre_peli,nombre_actxr,conocido_por,year_nacimiento,profesiones_string,premios
0,Redemption,Derek Burke,"[Redemption, Reversion]",N/A,Reparto,N/A
1,Redemption,July Smith,"[Redemption, A Fire in a Dovecot]",N/A,Reparto,N/A
2,Redemption,Adam Chambers,"[Skylar contra el monstruo, I <3 Vampires]",1983,"Reparto, Dirección, Producción",3 premios y 2 nominaciones
3,Redemption,Shaina Vorspan,"[Dollface, Morir de pie]",junio,"Reparto, Guion, Dirección",1 premio y 1 nominación
4,Redemption,Marco Neves,"[4 Horsemen: Apocalypse, Redemption]",N/A,"Reparto, Equipos adicionales","1,76 m"
...,...,...,...,...,...,...
1034,Leave,Ron Livingston,"[Expediente Warren: The Conjuring, Trabajo bas...",1967,"Reparto, Cámara y electricidad, Producción",3 nominaciones
1035,Leave,Bryan Cranston,"[Breaking Bad, Malcolm]",1956,"Reparto, Producción, Guion",59 premios y 106 nominaciones en total
1036,Leave,M.C. Gainey,"[Club Desmadre, Cerdos salvajes: Con un par......",1948,"Reparto, Producción",1 nominación
1037,Leave,Freddie Joe Farnsworth,"[Capitán América: El primer vengador, Guerra m...",1968,"Especialistas, Equipos adicionales, Reparto",N/A


In [8]:
#Lo guardamos pasándolo a csv
df_extraccion.to_csv("actorxs_datos.csv")